In [ ]:
import wfdb
import numpy as np
import os

def segment_data(folder_path, segment_length, step_size):
    segments = []
   
    for file in os.listdir(folder_path):
        if file.endswith('.dat'):
            file_name = file.split('.')[0]
            
            record = wfdb.rdrecord(os.path.join(folder_path, file_name))
            ecg_signal = record.p_signal

            # only process the first two ecg channels
            ecg_signal = ecg_signal[:, :2]

            ecg_signal = ecg_signal.astype(np.float32)

            for start in range(0, len(ecg_signal) - segment_length, step_size):
                end = start + segment_length
                segment = ecg_signal[start:end]
                segments.append(segment)
           
    return np.array(segments)

segment_length = 10000
step_size = 5000

sca_folder = './data/train_data/SCD' # data in 200 Hz
normal_folder = './data/train_data/normal' # data in 128 Hz

sca_segments = segment_data(sca_folder, segment_length, step_size) 
normal_segments = segment_data(normal_folder, segment_length, step_size) 

print(sca_segments.shape)
print(normal_segments.shape)


In [12]:
# normalize the data
sca_segments_normalized = sca_segments / np.max(np.abs(sca_segments), axis=1, keepdims=True)
normal_segments_normalized = normal_segments / np.max(np.abs(normal_segments), axis=1, keepdims=True)

In [ ]:
from sklearn.model_selection import train_test_split

#label data
sca_label = np.ones(len(sca_segments_normalized))
normal_label = np.zeros(len(normal_segments_normalized))

X_num_data_points = 70051
Y_num_data_points = 70051

X = np.concatenate([sca_segments_normalized[:X_num_data_points], normal_segments_normalized[:X_num_data_points]], axis=0)
Y = np.concatenate([sca_label[:Y_num_data_points], normal_label[:Y_num_data_points]], axis=0)

X = X.astype(np.float32)
Y = Y.astype(np.float32)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


def create_model(input_shape):
    model = models.Sequential()

    model.add(layers.Conv1D(32, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    
    model.add(layers.Conv1D(64, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model(input_shape=(10000, 2))  # Each sample has 10000 time steps and 2 channels

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))

loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
keras_file_path = './models/model_3/keras/'
h5_file_path = './models/model_3/h5/'
exported_model_file_path = './models/model_3/exported_model'
onnx_file_path = './models/model_3/onnx/'

os.makedirs(os.path.dirname(keras_file_path), exist_ok=True)
os.makedirs(os.path.dirname(h5_file_path), exist_ok=True)
os.makedirs(os.path.dirname(exported_model_file_path), exist_ok=True)
os.makedirs(os.path.dirname(onnx_file_path), exist_ok=True)

model.save(keras_file_path + 'scd_model.keras')
model.save(h5_file_path + 'scd_model.h5')
model.export(exported_model_file_path)
!python -m tf2onnx.convert --saved-model ./models/model_3/exported_model --output ./models/model_3/onnx/scd_model.onnx --opset 13